In [6]:
from pathlib import Path
from typing import cast

import polars as pl
import plotly.graph_objects as go
from pprint import pprint

_ = (
    pl.Config.set_tbl_hide_dataframe_shape(True),
    pl.Config.set_tbl_width_chars(1000),
    pl.Config.set_tbl_cols(-1),
)

In [7]:
def get_data(
    source: str | Path, sheet: str
) -> pl.DataFrame:
    return cast(
        pl.DataFrame,
        pl.read_excel(
            source=source,
            sheet_name=sheet,
        ),
    )

In [8]:
info = get_data(Path(Path.cwd().parent,"data","steepener_trade_2024-01-02.xlsx"), "info")
clean = get_data(Path(Path.cwd().parent,"data","steepener_trade_2024-01-02.xlsx"), "clean price")
dirty = get_data(Path(Path.cwd().parent,"data","steepener_trade_2024-01-02.xlsx"), "dirty price")
duration = get_data(Path(Path.cwd().parent,"data","steepener_trade_2024-01-02.xlsx"), "duration")

print(info)
print(clean)
print(dirty)
print(duration)

┌───────────┬──────┬────────────┬────────────┬───────────────┬──────────┬──────────────────┬──────────┬────────────┬────────────┬────────────┬─────────────┬─────────────┬──────────┬─────────────┬──────────┐
│ KYTREASNO ┆ type ┆ quote date ┆ issue date ┆ maturity date ┆ ttm      ┆ accrual fraction ┆ cpn rate ┆ bid        ┆ ask        ┆ price      ┆ accrued int ┆ dirty price ┆ ytm      ┆ total size  ┆ duration │
│ ---       ┆ ---  ┆ ---        ┆ ---        ┆ ---           ┆ ---      ┆ ---              ┆ ---      ┆ ---        ┆ ---        ┆ ---        ┆ ---         ┆ ---         ┆ ---      ┆ ---         ┆ ---      │
│ i64       ┆ str  ┆ date       ┆ date       ┆ date          ┆ f64      ┆ f64              ┆ f64      ┆ f64        ┆ f64        ┆ f64        ┆ f64         ┆ f64         ┆ f64      ┆ i64         ┆ f64      │
╞═══════════╪══════╪════════════╪════════════╪═══════════════╪══════════╪══════════════════╪══════════╪════════════╪════════════╪════════════╪═════════════╪═════════════╪══

# 1.1

Long $50M notional in the 2-year, short the 10-year such that net dollar duration = 0.
For a duration-neutral portfolio:

$$\text{Dollar Duration} = \text{Duration} \times \text{Price} \times \text{Number of Bonds}$$
$$D_{2y} \cdot P_{2y} \cdot N_{2y} + D_{10y} \cdot P_{10y} \cdot N_{10y} = 0$$

Solving for $N_{10y}$:
$$N_{10y} = -\frac{D_{2y} \cdot P_{2y}}{D_{10y} \cdot P_{10y}} \cdot N_{2y}$$

The hedge ratio is:
$$h = \frac{D_{2y} \cdot P_{2y}}{D_{10y} \cdot P_{10y}}$$

In [9]:
BOND_2Y = "207652"
BOND_10Y = "208147"
FACE_VALUE = 100
NOTIONAL_LONG = 50_000_000

first_date = dirty.select("quote date").row(0)[0]

p_2y = dirty.row(0, named=True)[BOND_2Y]
p_10y = dirty.row(0, named=True)[BOND_10Y]
d_2y = duration.row(0, named=True)[BOND_2Y]
d_10y = duration.row(0, named=True)[BOND_10Y]

n_2y = NOTIONAL_LONG / FACE_VALUE

hedge_ratio = (d_2y * p_2y) / (d_10y * p_10y)
n_10y = -hedge_ratio * n_2y

short_notional = -n_10y * p_10y

print(f"""Date:\t\t\t\t{first_date}
2yr price:\t\t\t{p_2y:.4f},\tduration: {d_2y:.4f}
10yr price:\t\t\t{p_10y:.4f},\tduration: {d_10y:.4f}
Hedge ratio:\t\t\t{hedge_ratio:.6f}
Number of 2yr bonds (long):\t{n_2y:,.0f}
Number of 10yr bonds (short):\t{n_10y:,.2f}
Dollar value of short position: ${short_notional:,.2f}""")

Date:				2023-11-09
2yr price:			90.9783,	duration: 2.1352
10yr price:			99.0000,	duration: 8.1737
Hedge ratio:			0.240060
Number of 2yr bonds (long):	500,000
Number of 10yr bonds (short):	-120,029.95
Dollar value of short position: $11,882,965.53


$$\underbrace{2.135 \times 90.98 \times 500{,}000}_{\text{2yr DD}} + \underbrace{8.174 \times 99.00 \times (-120{,}030)}_{\text{10yr DD}} \approx 0$$

# 1.2

In [10]:
positions = (
    dirty.select("quote date", BOND_2Y, BOND_10Y)
    .rename({BOND_2Y: "p_2y", BOND_10Y: "p_10y"})
    .join(
        duration.select("quote date", BOND_2Y, BOND_10Y)
        .rename({BOND_2Y: "d_2y", BOND_10Y: "d_10y"}),
        on="quote date",
    )
    .with_columns(
        n_2y=pl.lit(NOTIONAL_LONG / FACE_VALUE),
    )
    .with_columns(
        hedge_ratio=(pl.col("d_2y") * pl.col("p_2y")) / (pl.col("d_10y") * pl.col("p_10y")),
    )
    .with_columns(
        n_10y=-pl.col("hedge_ratio") * pl.col("n_2y"),
    )
)

fig = go.Figure()
fig.add_trace(go.Scatter(x=positions["quote date"], y=positions["hedge_ratio"], name="Hedge Ratio"))
fig.update_layout(title="Hedge Ratio Over Time", xaxis_title="Date", yaxis_title="Hedge Ratio")
fig.show()

fig = go.Figure()
fig.add_trace(go.Scatter(x=positions["quote date"], y=positions["n_2y"], name="2yr Bonds"))
fig.update_layout(title="Number of 2yr Bonds", xaxis_title="Date", yaxis_title="# Bonds")
fig.show()

fig = go.Figure()
fig.add_trace(go.Scatter(x=positions["quote date"], y=positions["n_10y"], name="10yr Bonds"))
fig.update_layout(title="Number of 10yr Bonds (Short)", xaxis_title="Date", yaxis_title="# Bonds")
fig.show()

print(positions.tail(1))

┌────────────┬────────┬────────────┬─────────┬──────────┬──────────┬─────────────┬───────────────┐
│ quote date ┆ p_2y   ┆ p_10y      ┆ d_2y    ┆ d_10y    ┆ n_2y     ┆ hedge_ratio ┆ n_10y         │
│ ---        ┆ ---    ┆ ---        ┆ ---     ┆ ---      ┆ ---      ┆ ---         ┆ ---           │
│ date       ┆ f64    ┆ f64        ┆ f64     ┆ f64      ┆ f64      ┆ f64         ┆ f64           │
╞════════════╪════════╪════════════╪═════════╪══════════╪══════════╪═════════════╪═══════════════╡
│ 2025-05-30 ┆ 97.949 ┆ 101.685377 ┆ 0.58808 ┆ 7.135339 ┆ 500000.0 ┆ 0.079389    ┆ -39694.738124 │
└────────────┴────────┴────────────┴─────────┴──────────┴──────────┴─────────────┴───────────────┘


# 1.3

$$\text{PnL}_{t+1} = N_{2y,t} \cdot \Delta P_{2y,t+1} + N_{10y,t} \cdot \Delta P_{10y,t+1}$$

In [11]:
pnl = (
    positions
    .with_columns(
        dp_2y=pl.col("p_2y").diff(),
        dp_10y=pl.col("p_10y").diff(),
    )
    .with_columns(
        n_2y_lag=pl.col("n_2y").shift(1),
        n_10y_lag=pl.col("n_10y").shift(1),
    )
    .with_columns(
        pnl_2y=pl.col("n_2y_lag") * pl.col("dp_2y"),
        pnl_10y=pl.col("n_10y_lag") * pl.col("dp_10y"),
    )
    .with_columns(
        pnl_net=pl.col("pnl_2y") + pl.col("pnl_10y"),
    )
)

print("First day PnL:")
pprint(pnl.select("quote date", "pnl_2y", "pnl_10y", "pnl_net").row(1, named=True))

print("\nLast day PnL:")
pprint(pnl.select("quote date", "pnl_2y", "pnl_10y", "pnl_net").row(-1, named=True))

First day PnL:
{'pnl_10y': 25318.818592902517,
 'pnl_2y': -19021.739130437254,
 'pnl_net': 6297.079462465263,
 'quote date': datetime.date(2023, 11, 10)}

Last day PnL:
{'pnl_10y': -11498.135002501662,
 'pnl_2y': 15615.85580110858,
 'pnl_net': 4117.720798606919,
 'quote date': datetime.date(2025, 5, 30)}


# 1.4

In [12]:
pnl_plot = pnl.with_columns(
    cum_pnl_2y=pl.col("pnl_2y").cum_sum(),
    cum_pnl_10y=pl.col("pnl_10y").cum_sum(),
    cum_pnl_net=pl.col("pnl_net").cum_sum(),
)

fig = go.Figure()
fig.add_trace(go.Scatter(x=pnl_plot["quote date"], y=pnl_plot["cum_pnl_2y"], mode="lines", name="2yr (Long)"))
fig.add_trace(go.Scatter(x=pnl_plot["quote date"], y=pnl_plot["cum_pnl_10y"], mode="lines", name="10yr (Short)"))
fig.add_trace(go.Scatter(x=pnl_plot["quote date"], y=pnl_plot["cum_pnl_net"], mode="lines", name="Net"))
fig.update_layout(
    title="Cumulative PnL - Steepener Trade",
    xaxis_title="Date",
    yaxis_title="Cumulative PnL ($)",
    hovermode="x unified",
)
fig.show()

print(f"Total cumulative PnL (no coupons): ${pnl_plot['cum_pnl_net'][-1]:,.2f}")

Total cumulative PnL (no coupons): $3,043,663.49


# 1.5

When a coupon is paid, accrued interest resets to zero, so clean price ≈ dirty price on the day *after* the coupon. The coupon payment per bond is coupon $\frac{\text{coupon rate}}{2} \times 100$ (semiannual, face value 100).

In [13]:
CPN_2Y = 0.375 / 100
CPN_10Y = 4.5 / 100
COUPON_PER_BOND_2Y = (CPN_2Y / 2) * FACE_VALUE
COUPON_PER_BOND_10Y = (CPN_10Y / 2) * FACE_VALUE

clean_df = clean.rename({BOND_2Y: "clean_2y", BOND_10Y: "clean_10y"})
dirty_df = dirty.rename({BOND_2Y: "dirty_2y", BOND_10Y: "dirty_10y"})

coupon_df = (
    clean_df
    .join(dirty_df, on="quote date")
    .with_columns(
        is_coupon_2y=(pl.col("clean_2y") - pl.col("dirty_2y")).abs() < 0.001,
        is_coupon_10y=(pl.col("clean_10y") - pl.col("dirty_10y")).abs() < 0.001,
    )
    .select("quote date", "is_coupon_2y", "is_coupon_10y")
)

pnl_with_coupons = (
    pnl
    .join(coupon_df, on="quote date")
    .with_columns(
        coupon_2y=pl.when(pl.col("is_coupon_2y")).then(pl.col("n_2y_lag") * COUPON_PER_BOND_2Y).otherwise(0.0),
        coupon_10y=pl.when(pl.col("is_coupon_10y")).then(pl.col("n_10y_lag") * COUPON_PER_BOND_10Y).otherwise(0.0),
    )
    .with_columns(
        pnl_net_with_coupons=pl.col("pnl_net") + pl.col("coupon_2y") + pl.col("coupon_10y"),
    )
    .with_columns(
        cum_pnl_net=pl.col("pnl_net").cum_sum(),
        cum_pnl_net_with_coupons=pl.col("pnl_net_with_coupons").cum_sum(),
    )
)

print(f"Cumulative PnL without coupons: ${pnl_with_coupons['cum_pnl_net'][-1]:,.2f}")
print(f"Cumulative PnL with coupons:    ${pnl_with_coupons['cum_pnl_net_with_coupons'][-1]:,.2f}")

Cumulative PnL without coupons: $3,043,663.49
Cumulative PnL with coupons:    $1,587,907.99


In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=pnl_with_coupons["quote date"], 
    y=pnl_with_coupons["cum_pnl_net"], 
    mode="lines", 
    name="Without Coupons"
))
fig.add_trace(go.Scatter(
    x=pnl_with_coupons["quote date"], 
    y=pnl_with_coupons["cum_pnl_net_with_coupons"], 
    mode="lines", 
    name="With Coupons"
))Í
fig.update_layout(
    title="Steepener Trade: Cumulative PnL",
    xaxis_title="Date",
    yaxis_title="Cumulative PnL ($)",
    hovermode="x unified",
)
fig.show()